In [49]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 336 kB in 1s (236 kB/s)
Reading package lists... Done


In [50]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [51]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

In [52]:
# 2. Create a temporary view of the DataFrame.
spark.sparkContext.addFile(url)
hs_revised = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

hs_revised.show()

hs_revised.createOrReplaceTempView("home_sales")

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [53]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
from pyspark.sql.functions import round
from pyspark.sql.functions import avg

average_prices_4b = spark.sql("""
    SELECT YEAR(Date) AS Year, ROUND(AVG(Price), 2) AS AveragePrice
    FROM home_sales
    WHERE Bedrooms = 4
    GROUP BY YEAR(Date)
    ORDER BY Year
""")
average_prices_4b.show()

+----+------------+
|Year|AveragePrice|
+----+------------+
|2019|    300263.7|
|2020|   298353.78|
|2021|   301819.44|
|2022|   296363.88|
+----+------------+



In [54]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
average_price_3b3b = spark.sql("""
    SELECT YEAR(Date) AS Year, ROUND(AVG(Price), 2) AS AveragePrice
    FROM home_sales
    WHERE Bedrooms = 3 AND Bathrooms = 3
    GROUP BY YEAR(Date)
    ORDER BY Year
""")
average_price_3b3b.show()

+----+------------+
|Year|AveragePrice|
+----+------------+
|2019|   287287.82|
|2020|   294204.16|
|2021|   294211.46|
|2022|   292725.69|
+----+------------+



In [55]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
average_price_3b3b_2fl_2000sqft = spark.sql("""
    SELECT YEAR(Date) AS Year, ROUND(AVG(Price), 2) AS AveragePrice
    FROM home_sales
    WHERE Bedrooms = 3 AND Bathrooms = 3 AND Floors = 2 AND sqft_lot >= 2000
    GROUP BY YEAR(Date)
    ORDER BY Year
""")
average_price_3b3b_2fl_2000sqft.show()


+----+------------+
|Year|AveragePrice|
+----+------------+
|2019|   282752.72|
|2020|   293176.41|
|2021|   290319.06|
|2022|   287162.18|
+----+------------+



In [56]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

average_price_350k = spark.sql("""
    SELECT view, ROUND(AVG(Price), 2) AS AveragePrice
    FROM home_sales
    WHERE Price >= 350000
    GROUP BY view
    ORDER BY view desc
""")


average_price_350k.show()

start_time = time.time()

print("--- %s seconds ---" % (time.time() - start_time))

+----+------------+
|view|AveragePrice|
+----+------------+
|  99|  1061201.42|
|  98|  1053739.33|
|  97|  1129040.15|
|  96|  1017815.92|
|  95|   1054325.6|
|  94|   1033536.2|
|  93|  1026006.06|
|  92|   970402.55|
|  91|  1137372.73|
|  90|  1062654.16|
|   9|   401393.34|
|  89|  1107839.15|
|  88|  1031719.35|
|  87|   1072285.2|
|  86|  1070444.25|
|  85|  1056336.74|
|  84|  1117233.13|
|  83|  1033965.93|
|  82|   1063498.0|
|  81|  1053472.79|
+----+------------+
only showing top 20 rows

--- 0.00010919570922851562 seconds ---


In [57]:
# 7. Cache the the temporary table home_sales.
spark.table('home_sales').cache()

DataFrame[id: string, date: string, date_built: string, price: string, bedrooms: string, bathrooms: string, sqft_living: string, sqft_lot: string, floors: string, waterfront: string, view: string]

In [58]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [59]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

average_price_350k_cachced = spark.sql("""
    SELECT view, ROUND(AVG(Price), 2) AS AveragePrice
    FROM home_sales
    WHERE Price >= 350000
    GROUP BY view
    ORDER BY view desc
""")

start_time = time.time()

average_price_350k_cachced.show()


print("--- %s seconds ---" % (time.time() - start_time))


+----+------------+
|view|AveragePrice|
+----+------------+
|  99|  1061201.42|
|  98|  1053739.33|
|  97|  1129040.15|
|  96|  1017815.92|
|  95|   1054325.6|
|  94|   1033536.2|
|  93|  1026006.06|
|  92|   970402.55|
|  91|  1137372.73|
|  90|  1062654.16|
|   9|   401393.34|
|  89|  1107839.15|
|  88|  1031719.35|
|  87|   1072285.2|
|  86|  1070444.25|
|  85|  1056336.74|
|  84|  1117233.13|
|  83|  1033965.93|
|  82|   1063498.0|
|  81|  1053472.79|
+----+------------+
only showing top 20 rows

--- 2.374770164489746 seconds ---


In [60]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
hs_revised.write.partitionBy("date_built").parquet("parq_home_sales", mode= "overwrite")

In [61]:
# 11. Read the parquet formatted data.
home_sales_partitioned = spark.read.parquet("parq_home_sales")

In [62]:
# 12. Create a temporary table for the parquet data.
home_sales_partitioned.createOrReplaceTempView('home_sales_partitioned')

In [63]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.

average_price_350k_partitioned = spark.sql("""
    SELECT view, ROUND(AVG(Price), 2) AS AveragePrice
    FROM home_sales_partitioned
    WHERE Price >= 350000
    GROUP BY view
    ORDER BY view desc
""")

start_time = time.time()

average_price_350k_partitioned.show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+------------+
|view|AveragePrice|
+----+------------+
|  99|  1061201.42|
|  98|  1053739.33|
|  97|  1129040.15|
|  96|  1017815.92|
|  95|   1054325.6|
|  94|   1033536.2|
|  93|  1026006.06|
|  92|   970402.55|
|  91|  1137372.73|
|  90|  1062654.16|
|   9|   401393.34|
|  89|  1107839.15|
|  88|  1031719.35|
|  87|   1072285.2|
|  86|  1070444.25|
|  85|  1056336.74|
|  84|  1117233.13|
|  83|  1033965.93|
|  82|   1063498.0|
|  81|  1053472.79|
+----+------------+
only showing top 20 rows

--- 1.0094280242919922 seconds ---


In [66]:
# 14. Uncache the home_sales temporary table.
spark.table("home_sales").unpersist()

DataFrame[id: string, date: string, date_built: string, price: string, bedrooms: string, bathrooms: string, sqft_living: string, sqft_lot: string, floors: string, waterfront: string, view: string]

In [67]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached('home_sales')

# Print the caching status
if is_cached:
    print("The home_sales temporary table is still cached.")
else:
    print("The home_sales temporary table is not cached.")

The home_sales temporary table is not cached.
